# Módulo 1: HTML: Requests y BeautifulSoup
## descargando la pagina (requests)
<img src='https://www.pagina12.com.ar/assets/media/logos/logo_pagina_12_n.svg?v=1.0.178' width=300></img>
En este módulo veremos cómo utilizar las bibliotecas `requests` y `bs4` para programar scrapers de sitios HTML. Nos propondremos armar un scraper de noticias del diario <a href='www.pagina12.com.ar'>Página 12</a>.

Supongamos que queremos leer el diario por internet. Lo primero que hacemos es abrir el navegador, escribir la URL del diario y apretar Enter para que aparezca la página del diario. Lo que ocurre en el momento en el que apretamos Enter es lo siguiente:
1. El navegador envía una solicitud a la URL pidiéndole información.
2. El servidor recibe la petición y procesa la respuesta.
3. El servidor envía la respuesta a la IP de la cual recibió la solicitud.
4. Nuestro navegador recibe la respuesta y la muestra **formateada** en pantalla.

Para hacer un scraper debemos hacer un programa que replique este flujo de forma automática y sistemática para luego extraer la información deseada de la respuesta. Utilizaremos `requests` para realizar peticiones y recibir las respuestas y `bs4` para *parsear* la respuesta y extraer la información.<br>
Te dejo unos links que tal vez te sean de utilidad:
- [Códigos de status HTTP](https://developer.mozilla.org/es/docs/Web/HTTP/Status)
- [Documentación de requests](https://requests.kennethreitz.org/en/master/)
- [Documentación de bs4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [1]:
import requests
import pandas as pd

In [11]:
URL= "https://www.pagina12.com.ar/"
p12 = requests.get(URL) # enviamos una peticion a la pagina web
p12.status_code # consultamos el status de la consult, un 200 es repuesta existosa

200

In [12]:
# print(p12.text) # nos regresara en formato de texto plano el contenido html de la pagina
# print(p12.content) # lo mismo que .text pero el contenido estara dentro de una etiqueta que dice el tipo de contenido
# ejemplo: b'<! la b indica que son bits 

### acceso a los headers
recordando el curso 53 sabemos que cuando se ejecuta el modelo cliente servidor, tanto en la request http como en la response http hay headers que dan metadatos hacia el servidor, como de regreso en la http response 

In [13]:
p12.headers # headers de la response http
p12.request.headers # headers de la request http

{'User-Agent': 'python-requests/2.27.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

En el header del request http 'User-Agent': 'python-requests/2.27.1' es un campo muy importante ya que le indica al servidor que la peticion probiene de un sistema automatizado de python, en ocaciones el server te bloqueara por esta variable 

In [35]:
print(p12.request.method) # nos indica el metodo que se uso para la peticion
print(p12.url) # nos muestra la url por donde se envia la response http, util cuando 

GET
https://www.pagina12.com.ar/


## Parsing Pagina (Beatiful Soup)


In [15]:
from bs4 import BeautifulSoup

In [16]:
sp = BeautifulSoup(p12.text, "lxml") # recibe el contenido html y el parser (codigo que separa el contenido, osea lxml)
# si "lxml" no funciona intenta 'html.parser' o si tienes linux pip3 install lxml 

In [25]:
#print(sp.prettify()) # nos regresara el contenido indentado, 
# ya que el DOM(document, object model) del html organiza en forma de arbol

<ul class="horizontal-list main-supplements hide-on-mobile"><li><span class="today">Hoy:</span></li><li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/suplementos/verano12">Verano12</a></li><li class=""><a href="https://www.pagina12.com.ar/suplementos/no">NO</a></li></ul>

In [26]:
sp.find("ul") # nos traera la 1er etiqueta(tag) ul
sp.find("ul", attrs={'class':'hot-sections'}) # regresa la tag ul con la class=hot-sections
secciones = sp.find("ul", attrs={'class':'main-sections'}).find_all('li')# # nos regresara un lista de todas las tag li de la busqueda previa
secciones

[<li class="p12-separator--right--blue"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>,
 <li class="p12-separator--right--blue"><a href="https://www.pagina12.com.ar/secciones/economia">Economía</a></li>,
 <li class="p12-separator--right--blue"><a href="https://www.pagina12.com.ar/secciones/sociedad">Sociedad</a></li>,
 <li class="no-separator-on-1040 p12-separator--right--blue"><a href="https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos">Espectáculos</a></li>,
 <li class="hide-on-1040 p12-separator--right--blue"><a href="https://www.pagina12.com.ar/secciones/deportes">Deportes</a></li>,
 <li class="hide-on-1040 p12-separator--right--blue"><a href="https://www.pagina12.com.ar/secciones/ciencia">Ciencia</a></li>,
 <li class="hide-on-1040"><a href="https://www.pagina12.com.ar/secciones/el-mundo">El mundo</a></li>]

In [30]:
seccion = secciones[0] # seleccionamos la primera tag li, un equivalente es secciones.a, secciones.find("a")
seccion 

<li class="p12-separator--right--blue"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>

### extrayendo informacion de las etiquetas
si recuerdas las clases de html de desarrollo web las etiquetas tienen: 
- atributos(id, class, urls) 
- texto 

In [36]:
# ahora podemos acceder mas facilmente a el contenido de la etiqueta li
print(seccion.a.get('href')) # asi obtenemos el ATRIBUTO link de la etiqueta a 
print(seccion.a.get_text())
print(seccion.text)

https://www.pagina12.com.ar/secciones/el-pais
El país
El país


In [40]:
[sec.text for sec in secciones] # para obtener todos los textos de las etiquetas li
links_secciones = [sec.a.get('href') for sec in secciones]  # para obtener todos los links de la etiquetas a 
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo']

#### el poder del scrapping
con la lista de enlaces ahora podriamos usar request para analizar cada pestaña, aqui inica el poder del scraping.
analizaremos la pagina de la seccion el pais

In [42]:
sec = requests.get(links_secciones[0])
print(sec.request.url)
print(sec.status_code)

'https://www.pagina12.com.ar/secciones/el-pais'

In [60]:
soup_seccion = BeautifulSoup(sec.text, 'lxml')
#print(soup_seccion.prettify())

si inspeccionas la página te daras cuenta que hay 3 diferentes titulos h2,h3,h4 que contienen la etiqueta "a" con los links a cada noticia de la seccion del pais. los 3 titulos comparten la peculiaridad de tener la clase title-list, con esto podemos extraer los lins de cada noticia en la pestaña. 
ten en consideracion que los links tienen esta forma:   
"/515409-alberto-fernandez-siempre-vamos-en-socorro-de-los-que-mas-ne"  
por lo que debemos añadirle la direccion de la pagina actual usando "sec.request.url" para que tengamos:  
"https://www.pagina12.com.ar/secciones/el-pais/515409-alberto-fernandez-siempre-vamos-en-socorro-de-los-que-mas-ne"

In [73]:
featured_article = soup_seccion.find_all(['h2','h3','h4'], # asi se seleccionan varias etiqeutas al mismo tiempo
                                        attrs={'class':'title-list'}) 
[sec.request.url + art.a.get('href') for art in featured_article]

['https://www.pagina12.com.ar/secciones/el-pais/515409-alberto-fernandez-siempre-vamos-en-socorro-de-los-que-mas-ne',
 'https://www.pagina12.com.ar/secciones/el-pais/515408-hurlingham-reclamos-cruzados-por-la-obra-de-un-hospital',
 'https://www.pagina12.com.ar/secciones/el-pais/515404-paritarias-la-pulseada-por-la-recuperacion-de-los-salarios',
 'https://www.pagina12.com.ar/secciones/el-pais/515373-nuevo-reclamo-para-que-alberto-fernandez-indulte-a-milagro-s',
 'https://www.pagina12.com.ar/secciones/el-pais/515399-organismos-de-derechos-humanos-apoyan-el-juicio-politico-a-l',
 'https://www.pagina12.com.ar/secciones/el-pais/515397-sesiones-extraordinarias-con-sorpresas-ampliacion-de-la-cort',
 'https://www.pagina12.com.ar/secciones/el-pais/515382-brasil-oficializo-su-regreso-a-la-celac',
 'https://www.pagina12.com.ar/secciones/el-pais/515366-martin-soria-impulsar-reformas-legislativas-no-socava-el-est',
 'https://www.pagina12.com.ar/secciones/el-pais/515348-los-27-proyectos-que-envia-el

para automatizar mas el proceso crearemos una funcion para que te regrese una lista a todas las noticias de una seccion

In [75]:
def get_link_from_section(section_url): # la funcion resivira el link a una de las secciones como: pais, deporte, etc
    sec = requests.get(section_url)
    soup_seccion = BeautifulSoup(sec.text, 'lxml')
    articles_title = soup_seccion.find_all(['h2','h3','h4'], attrs={'class':'title-list'}) 
    return [sec.request.url + link.a.get('href') for link in articles_title]

get_link_from_section(soup_seccion)

['https://www.pagina12.com.ar/secciones/el-pais/515409-alberto-fernandez-siempre-vamos-en-socorro-de-los-que-mas-ne',
 'https://www.pagina12.com.ar/secciones/el-pais/515408-hurlingham-reclamos-cruzados-por-la-obra-de-un-hospital',
 'https://www.pagina12.com.ar/secciones/el-pais/515404-paritarias-la-pulseada-por-la-recuperacion-de-los-salarios',
 'https://www.pagina12.com.ar/secciones/el-pais/515373-nuevo-reclamo-para-que-alberto-fernandez-indulte-a-milagro-s',
 'https://www.pagina12.com.ar/secciones/el-pais/515399-organismos-de-derechos-humanos-apoyan-el-juicio-politico-a-l',
 'https://www.pagina12.com.ar/secciones/el-pais/515397-sesiones-extraordinarias-con-sorpresas-ampliacion-de-la-cort',
 'https://www.pagina12.com.ar/secciones/el-pais/515382-brasil-oficializo-su-regreso-a-la-celac',
 'https://www.pagina12.com.ar/secciones/el-pais/515366-martin-soria-impulsar-reformas-legislativas-no-socava-el-est',
 'https://www.pagina12.com.ar/secciones/el-pais/515348-los-27-proyectos-que-envia-el

### manejo de errores 

11